In [1]:
!pip install networkx

In [16]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00


In [26]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 49.9 MB/s eta 0:00:00


### 1. Create a protein-protein interaction network

In [2]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2025-05-13 13:10:02--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 682638053 (651M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 651.01M  62.7MB/s    in 11s     

2025-05-13 13:10:13 (60.0 MB/s) - ‘uniprot_sprot.dat.gz’ saved [682638053/682638053]



In [3]:
!wget https://downloads.thebiogrid.org/Download/BioGRID/Latest-Release/BIOGRID-ORGANISM-LATEST.mitab.zip

--2025-05-13 13:10:13--  https://downloads.thebiogrid.org/Download/BioGRID/Latest-Release/BIOGRID-ORGANISM-LATEST.mitab.zip
Resolving downloads.thebiogrid.org (downloads.thebiogrid.org)... 173.255.198.187
Connecting to downloads.thebiogrid.org (downloads.thebiogrid.org)|173.255.198.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/download]
Saving to: ‘BIOGRID-ORGANISM-LATEST.mitab.zip’

BIOGRID-ORGANISM-LA     [             <=>    ] 171.47M  67.6MB/s    in 2.5s    

2025-05-13 13:10:16 (67.6 MB/s) - ‘BIOGRID-ORGANISM-LATEST.mitab.zip’ saved [179801435]



In [6]:
!unzip -p BIOGRID-ORGANISM-LATEST.mitab.zip BIOGRID-ORGANISM-Homo_sapiens-4.4.245.mitab.txt > BIOGRID-ORGANISM-Homo_sapiens-4.4.245.mitab.txt

In [7]:
!wget https://reactome.org/download/current/interactors/reactome.homo_sapiens.interactions.psi-mitab.txt

--2025-05-13 13:12:56--  https://reactome.org/download/current/interactors/reactome.homo_sapiens.interactions.psi-mitab.txt
Resolving reactome.org (reactome.org)... 100.25.71.177
Connecting to reactome.org (reactome.org)|100.25.71.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184341247 (176M) [text/plain]
Saving to: ‘reactome.homo_sapiens.interactions.psi-mitab.txt’

reactome.homo_sapie 100%[===================>] 175.80M  55.8MB/s    in 3.3s    

2025-05-13 13:12:59 (53.3 MB/s) - ‘reactome.homo_sapiens.interactions.psi-mitab.txt’ saved [184341247/184341247]



In [8]:
!git clone https://github.com/jjjk123/GBA-centrality.git

Cloning into 'GBA-centrality'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 527 (delta 67), reused 74 (delta 52), pack-reused 374 (from 1)
Receiving objects: 100% (527/527), 5.60 MiB | 11.50 MiB/s, done.
Resolving deltas: 100% (315/315), done.


In [9]:
!gunzip -c uniprot_sprot.dat.gz | python GBA-centrality/Interactome/uniprot_parser.py > uniprot_parsed.tsv

2025-05-13 13:13:12 INFO uniprot_parser.py: Starting to run
2025-05-13 13:20:31 INFO uniprot_parser.py: All done, completed successfully!


In [10]:
!python GBA-centrality/Interactome/interaction_parser.py --interaction_file BIOGRID-ORGANISM-Homo_sapiens*.mitab.txt --uniprot_file uniprot_parsed.tsv > interactions_Biogrid.tsv

2025-05-13 13:20:31 INFO interaction_parser.py: Parsing Uniprot file
2025-05-13 13:20:31 INFO interaction_parser.py: Parsing interaction file
2025-05-13 13:20:31 INFO interaction_parser.py: Done!


In [11]:
!python GBA-centrality/Interactome/interaction_parser.py --interaction_file reactome.homo_sapiens.interactions.psi-mitab.txt --uniprot_file uniprot_parsed.tsv > interactions_Reactome.tsv

2025-05-13 13:20:31 INFO interaction_parser.py: Parsing Uniprot file
2025-05-13 13:20:31 INFO interaction_parser.py: Parsing interaction file
2025-05-13 13:20:33 INFO interaction_parser.py: Done!


In [12]:
!python GBA-centrality/Interactome/build_interactome.py \
  --interactions_parsed_files interactions_Biogrid.tsv interactions_Reactome.tsv \
  --uniprot_file uniprot_parsed.tsv > interactome_human.sif

2025-05-13 13:20:34 INFO build_interactome.py: Parsing Uniprot file
2025-05-13 13:20:34 INFO build_interactome.py: Parsing interaction files
2025-05-13 13:20:35 INFO build_interactome.py: Removing protein hubs
2025-05-13 13:20:35 INFO build_interactome.py: Printing interactome
2025-05-13 13:20:35 INFO build_interactome.py: Done!


### 2. Test the first GNN: XGDAG

In [13]:
!git clone https://github.com/GiDeCarlo/XGDAG.git

Cloning into 'XGDAG'...
remote: Enumerating objects: 2841, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 2841 (delta 28), reused 50 (delta 15), pack-reused 2770 (from 1)
Receiving objects: 100% (2841/2841), 210.69 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (1165/1165), done.
Updating files: 100% (1360/1360), done.


In [30]:
import os
import yaml

with open("/content/XGDAG/CondaEnvs/xgdag_environment.yml") as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in environment_data["dependencies"]:
    if isinstance(dependency, dict):
      for lib in dependency['pip']:
        os.system(f"pip install {lib}")

In [ ]:
!python XGDAG/ComputeRankingScript.py

# error ModuleNotFoundError: No module named 'SubgraphXshapley'

Traceback (most recent call last):
  File "/content/XGDAG/ComputeRankingScript.py", line 4, in <module>
    from GDARanking import predict_candidate_genes
  File "/content/XGDAG/GDARanking.py", line 20, in <module>
    from SubgraphX import SubgraphX
  File "/content/XGDAG/SubgraphX/SubgraphX.py", line 21, in <module>
    from SubgraphXshapley import GnnNetsGC2valueFunc, GnnNetsNC2valueFunc, \
ModuleNotFoundError: No module named 'SubgraphXshapley'
